In [1]:
# using Plots
using Distributions, CSV, DataFrames, ProgressMeter
include("../../utils_sg.jl")

Define correlation matrix:

In [2]:
C = [1.0 0.5; 0.5 1.0]; Ψ = [0.65  0.3; -0.1 0.05]; R = [1.0 0.7; 0.7 1.0];
Π = [C Ψ; Ψ' R]; cholΠ=cholesky(Π).L
CT = [0.5 0.5 0.0 0.0; -0.5 0.5 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0];
invCT = inv(CT);

Parameters:

In [3]:
T = 1.0; r=0.01; R=0.07;
S0 = [100.0, 100.0]; 
v0 = [0.4, 0.3]; 
α = [0.65, 0.3]; 
β = [0.6, 0.07]; 
η = [0.9, 0.2]
θ₀ = [0.02, 0.3]
K = 100.0;
λ = [0.5, 0.5, 0.0, 0.0]

slvm1 = HestonSABRSLV(S0[1], v0[1], Ψ[1,1], r, β[1], η[1], α[1], θ₀[1]);
slvm2 = HestonSABRSLV(S0[2], v0[2], Ψ[2,2], r, β[2], η[2], α[2], θ₀[2]);

In [4]:
drift(x) = [slvm1.ω(abs(x[1]),abs(x[3])), slvm2.ω(abs(x[2]),abs(x[2])), slvm1.μ(abs(x[3])), slvm2.μ(abs(x[4]))]
diffusion(x) = Diagonal(
        [slvm1.m(abs(x[3]))*slvm1.Γ(abs(x[1])), slvm2.m(abs(x[4]))*slvm1.Γ(abs(x[2])), slvm1.σ(abs(x[3])), slvm2.σ(abs(x[4]))]
        )*cholesky(Π).L
driver(t, x, y, z) = (
    -r*max(y-z'*inv(diffusion(x))*x, 0.0)
    -R*min(y-z'*inv(diffusion(x))*x, 0.0)
    -z'*inv(diffusion(x))*drift(x)
)
terminal(x) = max(dot(λ, x)-K,0.0)
bsde = BSDE(
        T, [S0, v0], x->CT*drift(invCT*x), x->CT*diffusion(invCT*x), (t,x,y,z)->driver(t,invCT*x, y, z), x->terminal(invCT*x)
    );

### Experiment

In [5]:
d = 4
### Parameters for Tavella-Randall grids
g=1.0
g₁=g
g₂=g

lefts = [0.0, 0.0, 0.0, 0.0];
rights = [200.0, 200.0, 0.8, 0.6];
centers = [100.0, 100.0, 0.4, 0.3];
scaled_centers=(centers.-(rights.+lefts)./2.0)./((rights.-lefts)./2.0);

lefts_ct = [51.0, -49.0, 0.01, 0.01]
rights_ct = [149.0, 49.0, 0.79, 0.59]
centers_ct = CT*centers
scaled_centers_ct = (centers_ct.-(rights_ct.+lefts_ct)./2.0)./((rights_ct.-lefts_ct)./2.0);

In [6]:
function solveProb(mind, bsde, Nₜ, scheme)
    d = length(mind)
    
    _p = [rescale(eq_nodes(mind[dim]), scaled_centers_ct[dim], lefts_ct[dim], rights_ct[dim]) for dim in 1:d]
    Δₗ = [pp[2]-pp[1] for pp in _p]
    Δᵣ = [pp[end]-pp[end-1] for pp in _p]
    Nₗ = [length(pp)÷2 for pp in _p]
    Nᵣ = [length(pp)÷2 for pp in _p]

    grids = Array{AbstractGrid,1}(undef, d)
    for dim in 1:d
        if dim == 1
            grids[dim] = TavellaRandallGrid(
                g₁, 
                g₂, 
                _p[dim][1], 
                _p[dim][length(_p[dim])÷2+1], 
                _p[dim][end], 
                length(_p[dim])÷2, 
                length(_p[dim])÷2
            )
        else
            grids[dim] = Grid1D(_p[dim], length(_p[dim]), length(_p[dim])÷2, length(_p[dim])÷2)
        end
    end
    res = MethodOfLines(bsde, grids, Nₜ, scheme, true)
    return [res[1], grids]
end

solveProb (generic function with 1 method)

In [7]:
Nₜs = [10, 20, 50, 100, 200]
header = vcat(["q", "measurement_type"], string.(Nₜs))
qs = [5,6,7,8]
schemes = [
#     [LawsonEuler(krylov=true, m=100), true],
#     [NorsettEuler(krylov=true, m=100), true],
#     [ETDRK2(krylov=true, m=100), true],
#     [ETDRK3(krylov=true, m=100), true],
#     [ETDRK4(krylov=true, m=100), true],
    [HochOst4(krylov=true, m=100), true]
    # [DP5(), false],
    # [RadauIIA5(), false]
]

1-element Vector{Vector{Any}}:
 [HochOst4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]

In [8]:
dfs = Array{DataFrame, 1}(undef, length(schemes))
for (snum, attr) in enumerate(schemes)
    scheme = attr[1]
    EXPINT = attr[2]
    schemename = split(split(string(scheme), '{')[1], '(')[1]
    dfs[snum] = DataFrame([[],[],[],[],[],[],[]], header)
    for q in [11]
        print(string("q=", q,":\n")); flush(stdout)
        sol100 = zeros(Float64, length(Nₜs)); 
        runtimes = zeros(Float64, length(Nₜs))
        spind = sparse_indices(q, d)
        for (ind, Nₜ) in enumerate(Nₜs)
            print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
            exc_start = time()
            results = [solveProb(mind, bsde, Nₜ, scheme) for mind in spind]
            data = [dat[1] for dat in results]
            grids = [dat[2] for dat in results]
            sg_interp = SparseGridInterpolation(data, grids, spind, q, d, Nₜ, lefts, rights, centers)
            exc_stop = time()
            sol100[ind] = sg_interp.Interpolant(Nₜ+1, CT*[100.0,100.0,0.4,0.3])
            runtimes[ind] = exc_stop - exc_start
            print(string("Y0=", sol100[ind],":\n")); flush(stdout)
            print(string("Runtime[s]=", runtimes[ind],":\n")); flush(stdout)
        end
        row_100 = vcat([string(q), "Y0(100,0.4)"], string.(sol100)); push!(dfs[snum],row_100);
        row_run = vcat([string(q), "Runtime[s]"], string.(runtimes)); push!(dfs[snum],row_run);
    end
    dfs[snum] |> CSV.write(string(schemename, ".csv"))
end

q=11:
Nₜ=10:
Y0=7.430566992601248:
Runtime[s]=5965.276252985001:
Nₜ=20:
Y0=7.499859505201685:
Runtime[s]=10932.847093105316:
Nₜ=50:
Y0=7.516753556236795:
Runtime[s]=24935.392666101456:
Nₜ=100:
Y0=7.517130579269033:
Runtime[s]=48705.961517095566:
Nₜ=200:
Y0=7.517135635600779:
Runtime[s]=97810.96660494804:


In [ ]:
dfs = Array{DataFrame, 1}(undef, length(schemes))
for (snum, attr) in enumerate(schemes)
    scheme = attr[1]
    EXPINT = attr[2]
    schemename = split(split(string(scheme), '{')[1], '(')[1]
    dfs[snum] = DataFrame([[],[],[],[],[],[],[]], header)
    for q in [10,11]
        print(string("q=", q,":\n")); flush(stdout)
        sol100 = zeros(Float64, length(Nₜs)); 
        runtimes = zeros(Float64, length(Nₜs))
        spind = sparse_indices(q, d)
        for (ind, Nₜ) in enumerate(Nₜs)
            print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
            exc_start = time()
            results = [solveProb(mind, bsde, Nₜ, scheme) for mind in spind]
            data = [dat[1] for dat in results]
            grids = [dat[2] for dat in results]
            sg_interp = SparseGridInterpolation(data, grids, spind, q, d, Nₜ, lefts, rights, centers)
            exc_stop = time()
            sol100[ind] = sg_interp.Interpolant(Nₜ+1, CT*[100.0,100.0,0.4,0.3])
            runtimes[ind] = exc_stop - exc_start
            print(string("Y0=", sol100[ind],":\n")); flush(stdout)
            print(string("Runtime[s]=", runtimes[ind],":\n")); flush(stdout)
        end
        row_100 = vcat([string(q), "Y0(100,0.4)"], string.(sol100)); push!(dfs[snum],row_100);
        row_run = vcat([string(q), "Runtime[s]"], string.(runtimes)); push!(dfs[snum],row_run);
    end
    dfs[snum] |> CSV.write(string(schemename, ".csv"))
end

q=10:
Nₜ=10:
Y0=7.515217874558343:
Runtime[s]=838.1456301212311:
Nₜ=20:
Y0=7.516556705839124:
Runtime[s]=1487.505112171173:
Nₜ=50:
Y0=7.516579654831062:
Runtime[s]=3376.8531000614166:
Nₜ=100:
Y0=7.5165794781308914:
Runtime[s]=6542.38395690918:
Nₜ=200:
Y0=7.516579455261652:
Runtime[s]=12817.336641073227:
q=11:
Nₜ=10:


In [ ]:
dfs = Array{DataFrame, 1}(undef, length(schemes))
for (snum, attr) in enumerate(schemes)
    scheme = attr[1]
    EXPINT = attr[2]
    schemename = split(split(string(scheme), '{')[1], '(')[1]
    dfs[snum] = DataFrame([[],[],[],[],[],[],[]], header)
    for q in [8,9,10,11]
        print(string("q=", q,":\n")); flush(stdout)
        sol100 = zeros(Float64, length(Nₜs)); 
        runtimes = zeros(Float64, length(Nₜs))
        spind = sparse_indices(q, d)
        for (ind, Nₜ) in enumerate(Nₜs)
            print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
            exc_start = time()
            results = [solveProb(mind, bsde, Nₜ, scheme) for mind in spind]
            data = [dat[1] for dat in results]
            grids = [dat[2] for dat in results]
            sg_interp = SparseGridInterpolation(data, grids, spind, q, d, Nₜ, lefts, rights, centers)
            exc_stop = time()
            sol100[ind] = sg_interp.Interpolant(Nₜ+1, CT*[100.0,100.0,0.4,0.3])
            runtimes[ind] = exc_stop - exc_start
            print(string("Y0=", sol100[ind],":\n")); flush(stdout)
            print(string("Runtime[s]=", runtimes[ind],":\n")); flush(stdout)
        end
        row_100 = vcat([string(q), "Y0(100,0.4)"], string.(sol100)); push!(dfs[snum],row_100);
        row_run = vcat([string(q), "Runtime[s]"], string.(runtimes)); push!(dfs[snum],row_run);
    end
    dfs[snum] |> CSV.write(string(schemename, ".csv"))
end

q=8:
Nₜ=10:
Y0=7.500158040589735:
Runtime[s]=50.71484112739563:
Nₜ=20:
Y0=7.500131900683128:
Runtime[s]=57.89162492752075:
Nₜ=50:
Y0=7.500128496768227:
Runtime[s]=141.68213200569153:
Nₜ=100:
Y0=7.500128300000469:
Runtime[s]=276.37019205093384:
Nₜ=200:
Y0=7.500128275776776:
Runtime[s]=552.9504218101501:
q=9:
Nₜ=10:
Y0=7.5145741994326265:
Runtime[s]=119.64703392982483:
Nₜ=20:
Y0=7.5145492515036105:
Runtime[s]=195.73518705368042:
Nₜ=50:
Y0=7.514545416763401:
Runtime[s]=484.6726770401001:
Nₜ=100:
Y0=7.514545181527124:
Runtime[s]=931.0067451000214:
Nₜ=200:
Y0=7.514545152170882:
Runtime[s]=1904.4860670566559:
q=10:
Nₜ=10:
